# Notebook 2: Preparación de Dashboard Streamlit

**Fundación Teletón - Análisis de Satisfacción de Benefactores**

---

## Objetivo

Este notebook prepara:
- Funciones reutilizables para calcular KPIs
- Funciones de visualización con la paleta de Teletón
- Agregaciones de datos necesarias para el dashboard
- Componentes que se integrarán en la aplicación Streamlit

## Estructura del Dashboard Streamlit

1. **Header con KPIs principales**
2. **Perfil de Benefactores** (giro, estado, puesto)
3. **Calidad de Servicio** (13 dimensiones)
4. **Satisfacción y Lealtad** (NPS, transparencia)
5. **Análisis Temporal** (tendencias por antigüedad)

---

## 1. Importar Librerías y Configuración

In [ ]:
# Librerías estándar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# Paleta de colores Teletón
COLORES_TELETON = {
    'amarillo': '#F7C600',
    'morado_profundo': '#4B1F76',
    'morado_medio': '#7E3AA7',
    'magenta': '#D7268F',
    'azul': '#1A2A6C',
    'naranja': '#F39C12',
    'verde': '#2ECC71',
    'rojo': '#E74C3C',
    'gris_claro': '#F5F5F5',
    'gris_medio': '#7F8C8D',
    'gris_oscuro': '#2D3436'
}

# Paleta para gráficos
PALETA_PRINCIPAL = ['#F7C600', '#4B1F76', '#7E3AA7', '#D7268F', '#1A2A6C']
sns.set_palette(PALETA_PRINCIPAL)

print("✅ Librerías importadas")
print(f"📅 Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Cargar Dataset Limpio

In [ ]:
# Cargar datos limpios generados en Notebook 1
df = pd.read_csv('../datos/teleton_limpio.csv')

print(f"📊 Dataset cargado: {df.shape[0]} filas × {df.shape[1]} columnas")
print(f"\n📋 Columnas disponibles:")
print(df.columns.tolist())

## 3. Definir Variables por Categoría

In [ ]:
# 13 dimensiones de calidad de servicio (escala Likert 1-5)
VARS_CALIDAD_SERVICIO = [
    'apariencia_comportamiento',
    'claridad_procedimientos',
    'cumplimiento_horarios',
    'conocimiento_competencia',
    'claridad_primer_contacto',
    'rapidez_respuesta',
    'disposicion_ayuda',
    'flexibilidad',
    'actitud_comprensiva',
    'tiempo_dedicado',
    'preocupacion_situacion',
    'atencion_personalizada',
    'comunicacion_continua'
]

# 4 variables de satisfacción (escala 1-10)
VARS_SATISFACCION = [
    'satisfaccion_general',
    'nps',
    'calidad_percibida',
    'transparencia'
]

# Variables categóricas
VARS_CATEGORICAS = ['giro', 'puesto', 'estado']

# Variables numéricas
VARS_NUMERICAS = ['anos_benefactor']

# Variables calculadas
VARS_CALCULADAS = ['indice_calidad_servicio', 'nps_categoria', 'segmento_antiguedad']

print("✅ Variables clasificadas:")
print(f"   - Calidad de Servicio: {len(VARS_CALIDAD_SERVICIO)} variables")
print(f"   - Satisfacción: {len(VARS_SATISFACCION)} variables")
print(f"   - Categóricas: {len(VARS_CATEGORICAS)} variables")
print(f"   - Numéricas: {len(VARS_NUMERICAS)} variables")
print(f"   - Calculadas: {len(VARS_CALCULADAS)} variables")

## 4. Funciones para Calcular KPIs

Estas funciones serán reutilizadas en el dashboard de Streamlit.

In [ ]:
def calcular_kpi_satisfaccion_general(df):
    """
    KPI 1: Satisfacción General
    Promedio de la columna 'satisfaccion_general' (escala 1-10)
    """
    return df['satisfaccion_general'].mean()


def calcular_kpi_nps(df):
    """
    KPI 2: Net Promoter Score (NPS)
    Fórmula: (% Promotores - % Detractores) × 100
    - Promotores: NPS >= 9
    - Pasivos: 7 <= NPS <= 8
    - Detractores: NPS <= 6
    """
    total = len(df)
    promotores = (df['nps'] >= 9).sum()
    detractores = (df['nps'] <= 6).sum()
    
    nps = ((promotores - detractores) / total) * 100
    return nps


def calcular_kpi_calidad_servicio(df):
    """
    KPI 3: Índice de Calidad de Servicio
    Promedio de las 13 dimensiones de calidad (escala 1-5)
    """
    return df[VARS_CALIDAD_SERVICIO].mean().mean()


def calcular_kpi_transparencia(df):
    """
    KPI 4: Transparencia
    Promedio de la columna 'transparencia' (escala 1-10)
    """
    return df['transparencia'].mean()


def calcular_kpi_antiguedad_promedio(df):
    """
    KPI 5: Antigüedad Promedio
    Promedio de años como benefactor
    """
    return df['anos_benefactor'].mean()


def calcular_kpi_calidad_percibida(df):
    """
    KPI 6: Calidad Percibida
    Promedio de la columna 'calidad_percibida' (escala 1-10)
    """
    return df['calidad_percibida'].mean()


def calcular_todos_kpis(df):
    """
    Calcula todos los KPIs y devuelve un diccionario
    """
    kpis = {
        'satisfaccion_general': calcular_kpi_satisfaccion_general(df),
        'nps': calcular_kpi_nps(df),
        'indice_calidad_servicio': calcular_kpi_calidad_servicio(df),
        'transparencia': calcular_kpi_transparencia(df),
        'antiguedad_promedio': calcular_kpi_antiguedad_promedio(df),
        'calidad_percibida': calcular_kpi_calidad_percibida(df)
    }
    return kpis


# Probar las funciones
kpis = calcular_todos_kpis(df)
print("✅ KPIs Calculados:")
print(f"   1. Satisfacción General: {kpis['satisfaccion_general']:.2f} / 10")
print(f"   2. Net Promoter Score: {kpis['nps']:.1f}%")
print(f"   3. Índice Calidad Servicio: {kpis['indice_calidad_servicio']:.2f} / 5")
print(f"   4. Transparencia: {kpis['transparencia']:.2f} / 10")
print(f"   5. Antigüedad Promedio: {kpis['antiguedad_promedio']:.1f} años")
print(f"   6. Calidad Percibida: {kpis['calidad_percibida']:.2f} / 10")

## 5. Funciones de Segmentación NPS

In [ ]:
def segmentar_nps(df):
    """
    Calcula la distribución de promotores, pasivos y detractores
    Retorna DataFrame con conteos y porcentajes
    """
    total = len(df)
    
    promotores = (df['nps'] >= 9).sum()
    pasivos = ((df['nps'] >= 7) & (df['nps'] <= 8)).sum()
    detractores = (df['nps'] <= 6).sum()
    
    segmentos = pd.DataFrame({
        'Categoría': ['Promotores (9-10)', 'Pasivos (7-8)', 'Detractores (0-6)'],
        'Cantidad': [promotores, pasivos, detractores],
        'Porcentaje': [
            (promotores / total) * 100,
            (pasivos / total) * 100,
            (detractores / total) * 100
        ]
    })
    
    return segmentos


# Probar función
segmentos_nps = segmentar_nps(df)
print("✅ Segmentación NPS:")
print(segmentos_nps.to_string(index=False))

## 6. Funciones de Agregación por Segmento

In [ ]:
def agregar_por_giro(df):
    """
    Agrega métricas clave por tipo de giro empresarial
    """
    agregacion = df.groupby('giro').agg({
        'satisfaccion_general': 'mean',
        'nps': 'mean',
        'indice_calidad_servicio': 'mean',
        'anos_benefactor': 'mean',
        'giro': 'count'  # Para contar registros
    }).rename(columns={'giro': 'cantidad'})
    
    agregacion = agregacion.round(2)
    agregacion = agregacion.sort_values('satisfaccion_general', ascending=False)
    
    return agregacion.reset_index()


def agregar_por_estado(df):
    """
    Agrega métricas clave por estado
    """
    agregacion = df.groupby('estado').agg({
        'satisfaccion_general': 'mean',
        'nps': 'mean',
        'indice_calidad_servicio': 'mean',
        'estado': 'count'
    }).rename(columns={'estado': 'cantidad'})
    
    agregacion = agregacion.round(2)
    agregacion = agregacion.sort_values('cantidad', ascending=False)
    
    return agregacion.reset_index()


def agregar_por_antiguedad(df):
    """
    Agrega métricas por segmento de antigüedad
    """
    agregacion = df.groupby('segmento_antiguedad').agg({
        'satisfaccion_general': 'mean',
        'nps': 'mean',
        'indice_calidad_servicio': 'mean',
        'transparencia': 'mean',
        'segmento_antiguedad': 'count'
    }).rename(columns={'segmento_antiguedad': 'cantidad'})
    
    agregacion = agregacion.round(2)
    
    return agregacion.reset_index()


# Probar funciones
print("✅ Agregación por Giro:")
print(agregar_por_giro(df).head())
print("\n✅ Agregación por Estado (Top 5):")
print(agregar_por_estado(df).head())
print("\n✅ Agregación por Antigüedad:")
print(agregar_por_antiguedad(df))

## 7. Funciones de Visualización

Estas funciones crearán gráficos con la paleta de Teletón que serán usados en Streamlit.

### 7.1 Gráfico de KPIs (Métrica única)

In [ ]:
def crear_gauge_kpi(valor, titulo, valor_max=10, umbral_bueno=8, umbral_regular=6):
    """
    Crea un gráfico de "gauge" simple usando barras horizontales
    para mostrar un KPI con umbrales de color
    """
    # Determinar color según umbral
    if valor >= umbral_bueno:
        color = COLORES_TELETON['verde']
        categoria = 'Excelente'
    elif valor >= umbral_regular:
        color = COLORES_TELETON['amarillo']
        categoria = 'Bueno'
    else:
        color = COLORES_TELETON['rojo']
        categoria = 'Mejorable'
    
    fig, ax = plt.subplots(figsize=(8, 2))
    
    # Barra de fondo (gris)
    ax.barh([0], [valor_max], color=COLORES_TELETON['gris_claro'], height=0.5)
    
    # Barra de valor
    ax.barh([0], [valor], color=color, height=0.5)
    
    # Configuración
    ax.set_xlim(0, valor_max)
    ax.set_ylim(-0.5, 0.5)
    ax.set_yticks([])
    ax.set_xlabel('')
    ax.set_title(f"{titulo}: {valor:.2f} / {valor_max} ({categoria})", 
                 fontsize=14, fontweight='bold', color=COLORES_TELETON['morado_profundo'])
    
    # Texto del valor
    ax.text(valor/2, 0, f'{valor:.2f}', 
            ha='center', va='center', fontsize=16, fontweight='bold', color='white')
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    
    plt.tight_layout()
    return fig


# Probar visualización
fig = crear_gauge_kpi(
    valor=kpis['satisfaccion_general'],
    titulo='Satisfacción General',
    valor_max=10,
    umbral_bueno=8,
    umbral_regular=6
)
plt.show()
print("✅ Función crear_gauge_kpi() probada")

### 7.2 Gráfico de Distribución NPS

In [ ]:
def crear_grafico_nps(df):
    """
    Crea gráfico de barras horizontales con la distribución de NPS
    """
    segmentos = segmentar_nps(df)
    
    # Colores por categoría
    colores_nps = [COLORES_TELETON['verde'], COLORES_TELETON['amarillo'], COLORES_TELETON['rojo']]
    
    fig, ax = plt.subplots(figsize=(10, 5))
    
    bars = ax.barh(segmentos['Categoría'], segmentos['Porcentaje'], color=colores_nps)
    
    # Añadir etiquetas de porcentaje
    for i, (bar, cantidad, porcentaje) in enumerate(zip(bars, segmentos['Cantidad'], segmentos['Porcentaje'])):
        ax.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, 
                f'{porcentaje:.1f}% (n={cantidad})',
                va='center', fontsize=11, fontweight='bold')
    
    ax.set_xlabel('Porcentaje (%)', fontsize=12, fontweight='bold')
    ax.set_title('Distribución del Net Promoter Score (NPS)', 
                 fontsize=14, fontweight='bold', color=COLORES_TELETON['morado_profundo'])
    ax.set_xlim(0, max(segmentos['Porcentaje']) + 10)
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    plt.tight_layout()
    return fig


# Probar visualización
fig = crear_grafico_nps(df)
plt.show()
print("✅ Función crear_grafico_nps() probada")

### 7.3 Gráfico de Calidad de Servicio (13 dimensiones)

In [ ]:
def crear_grafico_calidad_servicio(df):
    """
    Crea gráfico de barras horizontales con las 13 dimensiones de calidad
    ordenadas de mayor a menor puntuación
    """
    # Calcular promedios
    promedios = df[VARS_CALIDAD_SERVICIO].mean().sort_values(ascending=True)
    
    # Nombres más legibles (primeras palabras)
    nombres_cortos = [
        'Apariencia y comportamiento',
        'Claridad de procedimientos',
        'Cumplimiento de horarios',
        'Conocimiento y competencia',
        'Claridad en primer contacto',
        'Rapidez de respuesta',
        'Disposición para ayudar',
        'Flexibilidad',
        'Actitud comprensiva',
        'Tiempo dedicado',
        'Preocupación por situación',
        'Atención personalizada',
        'Comunicación continua'
    ]
    
    # Crear diccionario de mapeo
    mapeo_nombres = dict(zip(VARS_CALIDAD_SERVICIO, nombres_cortos))
    promedios.index = promedios.index.map(mapeo_nombres)
    
    # Crear figura
    fig, ax = plt.subplots(figsize=(10, 8))
    
    bars = ax.barh(promedios.index, promedios.values, color=COLORES_TELETON['morado_medio'])
    
    # Añadir valores al final de cada barra
    for bar in bars:
        width = bar.get_width()
        ax.text(width + 0.05, bar.get_y() + bar.get_height()/2,
                f'{width:.2f}',
                va='center', fontsize=10, fontweight='bold')
    
    # Línea de referencia en la media general
    media_general = promedios.mean()
    ax.axvline(media_general, color=COLORES_TELETON['amarillo'], 
               linestyle='--', linewidth=2, label=f'Media general: {media_general:.2f}')
    
    ax.set_xlabel('Puntuación Promedio (escala 1-5)', fontsize=12, fontweight='bold')
    ax.set_title('Evaluación de las 13 Dimensiones de Calidad de Servicio', 
                 fontsize=14, fontweight='bold', color=COLORES_TELETON['morado_profundo'])
    ax.set_xlim(0, 5.5)
    ax.legend(loc='lower right')
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    plt.tight_layout()
    return fig


# Probar visualización
fig = crear_grafico_calidad_servicio(df)
plt.show()
print("✅ Función crear_grafico_calidad_servicio() probada")

### 7.4 Gráfico de Distribución por Giro

In [ ]:
def crear_grafico_por_giro(df, metrica='satisfaccion_general', top_n=10):
    """
    Crea gráfico de barras con la métrica promedio por giro empresarial
    Muestra solo los top_n giros con más registros
    """
    # Contar y filtrar top giros
    conteo_giros = df['giro'].value_counts().head(top_n).index
    df_filtrado = df[df['giro'].isin(conteo_giros)]
    
    # Calcular promedios
    promedios = df_filtrado.groupby('giro')[metrica].mean().sort_values(ascending=False)
    
    # Crear figura
    fig, ax = plt.subplots(figsize=(12, 6))
    
    bars = ax.bar(range(len(promedios)), promedios.values, color=COLORES_TELETON['morado_profundo'])
    
    # Configurar etiquetas
    ax.set_xticks(range(len(promedios)))
    ax.set_xticklabels(promedios.index, rotation=45, ha='right')
    
    # Añadir valores encima de las barras
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, height + 0.1,
                f'{height:.2f}',
                ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # Títulos
    titulo_metrica = metrica.replace('_', ' ').title()
    ax.set_ylabel(titulo_metrica, fontsize=12, fontweight='bold')
    ax.set_title(f'{titulo_metrica} por Giro Empresarial (Top {top_n})', 
                 fontsize=14, fontweight='bold', color=COLORES_TELETON['morado_profundo'])
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    plt.tight_layout()
    return fig


# Probar visualización
fig = crear_grafico_por_giro(df, metrica='satisfaccion_general', top_n=10)
plt.show()
print("✅ Función crear_grafico_por_giro() probada")

### 7.5 Gráfico Geográfico (Estados)

In [ ]:
def crear_grafico_geografico(df, top_n=10):
    """
    Crea gráfico de barras con cantidad de benefactores por estado
    """
    conteo_estados = df['estado'].value_counts().head(top_n)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    bars = ax.barh(conteo_estados.index, conteo_estados.values, color=COLORES_TELETON['azul'])
    
    # Añadir etiquetas
    for bar in bars:
        width = bar.get_width()
        ax.text(width + 1, bar.get_y() + bar.get_height()/2,
                f'{int(width)}',
                va='center', fontsize=10, fontweight='bold')
    
    ax.set_xlabel('Cantidad de Benefactores', fontsize=12, fontweight='bold')
    ax.set_title(f'Distribución Geográfica de Benefactores (Top {top_n} Estados)', 
                 fontsize=14, fontweight='bold', color=COLORES_TELETON['morado_profundo'])
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    plt.tight_layout()
    return fig


# Probar visualización
fig = crear_grafico_geografico(df, top_n=10)
plt.show()
print("✅ Función crear_grafico_geografico() probada")

### 7.6 Gráfico de Tendencia por Antigüedad

In [ ]:
def crear_grafico_antiguedad(df):
    """
    Crea gráfico de líneas mostrando cómo varían las métricas según antigüedad
    """
    # Ordenar categorías de antigüedad
    orden_antiguedad = ['Nuevo (0-3 años)', 'Establecido (4-7 años)', 
                        'Veterano (8-15 años)', 'Histórico (15+ años)']
    
    agregacion = df.groupby('segmento_antiguedad').agg({
        'satisfaccion_general': 'mean',
        'nps': 'mean',
        'indice_calidad_servicio': 'mean'
    }).reindex(orden_antiguedad)
    
    # Normalizar a escala 0-10 para comparación visual
    agregacion['indice_calidad_servicio_normalizado'] = agregacion['indice_calidad_servicio'] * 2
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    x = range(len(agregacion))
    
    ax.plot(x, agregacion['satisfaccion_general'], marker='o', linewidth=2.5,
            color=COLORES_TELETON['morado_profundo'], label='Satisfacción General (0-10)')
    
    ax.plot(x, agregacion['indice_calidad_servicio_normalizado'], marker='s', linewidth=2.5,
            color=COLORES_TELETON['morado_medio'], label='Calidad de Servicio (normalizado 0-10)')
    
    ax.plot(x, agregacion['nps'], marker='^', linewidth=2.5,
            color=COLORES_TELETON['amarillo'], label='NPS Promedio (0-10)')
    
    # Configuración
    ax.set_xticks(x)
    ax.set_xticklabels(agregacion.index, rotation=15, ha='right')
    ax.set_ylabel('Puntuación Promedio', fontsize=12, fontweight='bold')
    ax.set_title('Evolución de Métricas según Antigüedad del Benefactor', 
                 fontsize=14, fontweight='bold', color=COLORES_TELETON['morado_profundo'])
    ax.set_ylim(0, 10)
    ax.legend(loc='best', fontsize=10)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    plt.tight_layout()
    return fig


# Probar visualización
fig = crear_grafico_antiguedad(df)
plt.show()
print("✅ Función crear_grafico_antiguedad() probada")

### 7.7 Heatmap de Correlaciones

In [ ]:
def crear_heatmap_correlacion(df):
    """
    Crea heatmap de correlaciones entre variables de satisfacción y calidad de servicio
    """
    # Seleccionar variables clave
    vars_analisis = VARS_SATISFACCION + ['indice_calidad_servicio']
    
    # Calcular matriz de correlación
    correlacion = df[vars_analisis].corr()
    
    # Crear figura
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Heatmap con paleta personalizada
    sns.heatmap(correlacion, 
                annot=True, 
                fmt='.2f',
                cmap='YlGnBu',
                square=True,
                linewidths=1,
                cbar_kws={'label': 'Coeficiente de Correlación'},
                ax=ax)
    
    # Títulos más legibles
    labels_legibles = [
        'Satisfacción General',
        'NPS',
        'Calidad Percibida',
        'Transparencia',
        'Índice Calidad Servicio'
    ]
    
    ax.set_xticklabels(labels_legibles, rotation=45, ha='right')
    ax.set_yticklabels(labels_legibles, rotation=0)
    
    ax.set_title('Matriz de Correlación: Satisfacción y Calidad de Servicio', 
                 fontsize=14, fontweight='bold', color=COLORES_TELETON['morado_profundo'], pad=20)
    
    plt.tight_layout()
    return fig


# Probar visualización
fig = crear_heatmap_correlacion(df)
plt.show()
print("✅ Función crear_heatmap_correlacion() probada")

## 8. Exportar Agregaciones para Streamlit

In [ ]:
# Crear carpeta para datos procesados si no existe
import os
os.makedirs('../streamlit/data', exist_ok=True)

# Exportar agregaciones
agregar_por_giro(df).to_csv('../streamlit/data/agregacion_giro.csv', index=False)
agregar_por_estado(df).to_csv('../streamlit/data/agregacion_estado.csv', index=False)
agregar_por_antiguedad(df).to_csv('../streamlit/data/agregacion_antiguedad.csv', index=False)
segmentar_nps(df).to_csv('../streamlit/data/segmentos_nps.csv', index=False)

# Exportar KPIs
kpis_df = pd.DataFrame([kpis])
kpis_df.to_csv('../streamlit/data/kpis.csv', index=False)

print("✅ Archivos exportados a streamlit/data/:")
print("   - agregacion_giro.csv")
print("   - agregacion_estado.csv")
print("   - agregacion_antiguedad.csv")
print("   - segmentos_nps.csv")
print("   - kpis.csv")

## 9. Guardar Funciones en Módulo Python

Exportaremos todas las funciones a un archivo `.py` que será importado por la app de Streamlit.

In [ ]:
codigo_modulo = '''
"""
Módulo de utilidades para el Dashboard de Fundación Teletón
Contiene funciones para calcular KPIs, agregaciones y crear visualizaciones
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Paleta de colores Teletón
COLORES_TELETON = {
    'amarillo': '#F7C600',
    'morado_profundo': '#4B1F76',
    'morado_medio': '#7E3AA7',
    'magenta': '#D7268F',
    'azul': '#1A2A6C',
    'naranja': '#F39C12',
    'verde': '#2ECC71',
    'rojo': '#E74C3C',
    'gris_claro': '#F5F5F5',
    'gris_medio': '#7F8C8D',
    'gris_oscuro': '#2D3436'
}

PALETA_PRINCIPAL = ['#F7C600', '#4B1F76', '#7E3AA7', '#D7268F', '#1A2A6C']

# Variables por categoría
VARS_CALIDAD_SERVICIO = [
    'apariencia_comportamiento', 'claridad_procedimientos', 'cumplimiento_horarios',
    'conocimiento_competencia', 'claridad_primer_contacto', 'rapidez_respuesta',
    'disposicion_ayuda', 'flexibilidad', 'actitud_comprensiva',
    'tiempo_dedicado', 'preocupacion_situacion', 'atencion_personalizada',
    'comunicacion_continua'
]

VARS_SATISFACCION = ['satisfaccion_general', 'nps', 'calidad_percibida', 'transparencia']


# ===== FUNCIONES DE CÁLCULO DE KPIs =====

def calcular_kpi_satisfaccion_general(df):
    return df['satisfaccion_general'].mean()


def calcular_kpi_nps(df):
    total = len(df)
    promotores = (df['nps'] >= 9).sum()
    detractores = (df['nps'] <= 6).sum()
    return ((promotores - detractores) / total) * 100


def calcular_kpi_calidad_servicio(df):
    return df[VARS_CALIDAD_SERVICIO].mean().mean()


def calcular_kpi_transparencia(df):
    return df['transparencia'].mean()


def calcular_kpi_antiguedad_promedio(df):
    return df['anos_benefactor'].mean()


def calcular_kpi_calidad_percibida(df):
    return df['calidad_percibida'].mean()


def calcular_todos_kpis(df):
    return {
        'satisfaccion_general': calcular_kpi_satisfaccion_general(df),
        'nps': calcular_kpi_nps(df),
        'indice_calidad_servicio': calcular_kpi_calidad_servicio(df),
        'transparencia': calcular_kpi_transparencia(df),
        'antiguedad_promedio': calcular_kpi_antiguedad_promedio(df),
        'calidad_percibida': calcular_kpi_calidad_percibida(df)
    }


# ===== FUNCIONES DE SEGMENTACIÓN =====

def segmentar_nps(df):
    total = len(df)
    promotores = (df['nps'] >= 9).sum()
    pasivos = ((df['nps'] >= 7) & (df['nps'] <= 8)).sum()
    detractores = (df['nps'] <= 6).sum()
    
    return pd.DataFrame({
        'Categoría': ['Promotores (9-10)', 'Pasivos (7-8)', 'Detractores (0-6)'],
        'Cantidad': [promotores, pasivos, detractores],
        'Porcentaje': [
            (promotores / total) * 100,
            (pasivos / total) * 100,
            (detractores / total) * 100
        ]
    })


# ===== FUNCIONES DE AGREGACIÓN =====

def agregar_por_giro(df):
    agregacion = df.groupby('giro').agg({
        'satisfaccion_general': 'mean',
        'nps': 'mean',
        'indice_calidad_servicio': 'mean',
        'anos_benefactor': 'mean',
        'giro': 'count'
    }).rename(columns={'giro': 'cantidad'})
    agregacion = agregacion.round(2)
    return agregacion.sort_values('satisfaccion_general', ascending=False).reset_index()


def agregar_por_estado(df):
    agregacion = df.groupby('estado').agg({
        'satisfaccion_general': 'mean',
        'nps': 'mean',
        'indice_calidad_servicio': 'mean',
        'estado': 'count'
    }).rename(columns={'estado': 'cantidad'})
    agregacion = agregacion.round(2)
    return agregacion.sort_values('cantidad', ascending=False).reset_index()


def agregar_por_antiguedad(df):
    agregacion = df.groupby('segmento_antiguedad').agg({
        'satisfaccion_general': 'mean',
        'nps': 'mean',
        'indice_calidad_servicio': 'mean',
        'transparencia': 'mean',
        'segmento_antiguedad': 'count'
    }).rename(columns={'segmento_antiguedad': 'cantidad'})
    return agregacion.round(2).reset_index()


# ===== FUNCIONES DE VISUALIZACIÓN =====

def crear_gauge_kpi(valor, titulo, valor_max=10, umbral_bueno=8, umbral_regular=6):
    if valor >= umbral_bueno:
        color = COLORES_TELETON['verde']
        categoria = 'Excelente'
    elif valor >= umbral_regular:
        color = COLORES_TELETON['amarillo']
        categoria = 'Bueno'
    else:
        color = COLORES_TELETON['rojo']
        categoria = 'Mejorable'
    
    fig, ax = plt.subplots(figsize=(8, 2))
    ax.barh([0], [valor_max], color=COLORES_TELETON['gris_claro'], height=0.5)
    ax.barh([0], [valor], color=color, height=0.5)
    
    ax.set_xlim(0, valor_max)
    ax.set_ylim(-0.5, 0.5)
    ax.set_yticks([])
    ax.set_title(f"{titulo}: {valor:.2f} / {valor_max} ({categoria})", 
                 fontsize=14, fontweight='bold', color=COLORES_TELETON['morado_profundo'])
    
    ax.text(valor/2, 0, f'{valor:.2f}', 
            ha='center', va='center', fontsize=16, fontweight='bold', color='white')
    
    for spine in ['top', 'right', 'left']:
        ax.spines[spine].set_visible(False)
    
    plt.tight_layout()
    return fig


def crear_grafico_nps(df):
    segmentos = segmentar_nps(df)
    colores_nps = [COLORES_TELETON['verde'], COLORES_TELETON['amarillo'], COLORES_TELETON['rojo']]
    
    fig, ax = plt.subplots(figsize=(10, 5))
    bars = ax.barh(segmentos['Categoría'], segmentos['Porcentaje'], color=colores_nps)
    
    for bar, cantidad, porcentaje in zip(bars, segmentos['Cantidad'], segmentos['Porcentaje']):
        ax.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, 
                f'{porcentaje:.1f}% (n={cantidad})',
                va='center', fontsize=11, fontweight='bold')
    
    ax.set_xlabel('Porcentaje (%)', fontsize=12, fontweight='bold')
    ax.set_title('Distribución del Net Promoter Score (NPS)', 
                 fontsize=14, fontweight='bold', color=COLORES_TELETON['morado_profundo'])
    ax.set_xlim(0, max(segmentos['Porcentaje']) + 10)
    
    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)
    
    plt.tight_layout()
    return fig


def crear_grafico_calidad_servicio(df):
    promedios = df[VARS_CALIDAD_SERVICIO].mean().sort_values(ascending=True)
    
    nombres_cortos = [
        'Apariencia y comportamiento', 'Claridad de procedimientos',
        'Cumplimiento de horarios', 'Conocimiento y competencia',
        'Claridad en primer contacto', 'Rapidez de respuesta',
        'Disposición para ayudar', 'Flexibilidad',
        'Actitud comprensiva', 'Tiempo dedicado',
        'Preocupación por situación', 'Atención personalizada',
        'Comunicación continua'
    ]
    
    mapeo = dict(zip(VARS_CALIDAD_SERVICIO, nombres_cortos))
    promedios.index = promedios.index.map(mapeo)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    bars = ax.barh(promedios.index, promedios.values, color=COLORES_TELETON['morado_medio'])
    
    for bar in bars:
        width = bar.get_width()
        ax.text(width + 0.05, bar.get_y() + bar.get_height()/2,
                f'{width:.2f}', va='center', fontsize=10, fontweight='bold')
    
    media = promedios.mean()
    ax.axvline(media, color=COLORES_TELETON['amarillo'], 
               linestyle='--', linewidth=2, label=f'Media: {media:.2f}')
    
    ax.set_xlabel('Puntuación (1-5)', fontsize=12, fontweight='bold')
    ax.set_title('13 Dimensiones de Calidad de Servicio', 
                 fontsize=14, fontweight='bold', color=COLORES_TELETON['morado_profundo'])
    ax.set_xlim(0, 5.5)
    ax.legend(loc='lower right')
    
    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)
    
    plt.tight_layout()
    return fig


def crear_grafico_por_giro(df, metrica='satisfaccion_general', top_n=10):
    conteo_giros = df['giro'].value_counts().head(top_n).index
    df_filtrado = df[df['giro'].isin(conteo_giros)]
    promedios = df_filtrado.groupby('giro')[metrica].mean().sort_values(ascending=False)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    bars = ax.bar(range(len(promedios)), promedios.values, color=COLORES_TELETON['morado_profundo'])
    
    ax.set_xticks(range(len(promedios)))
    ax.set_xticklabels(promedios.index, rotation=45, ha='right')
    
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, height + 0.1,
                f'{height:.2f}', ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    titulo = metrica.replace('_', ' ').title()
    ax.set_ylabel(titulo, fontsize=12, fontweight='bold')
    ax.set_title(f'{titulo} por Giro (Top {top_n})', 
                 fontsize=14, fontweight='bold', color=COLORES_TELETON['morado_profundo'])
    
    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)
    
    plt.tight_layout()
    return fig


def crear_grafico_geografico(df, top_n=10):
    conteo = df['estado'].value_counts().head(top_n)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    bars = ax.barh(conteo.index, conteo.values, color=COLORES_TELETON['azul'])
    
    for bar in bars:
        width = bar.get_width()
        ax.text(width + 1, bar.get_y() + bar.get_height()/2,
                f'{int(width)}', va='center', fontsize=10, fontweight='bold')
    
    ax.set_xlabel('Cantidad', fontsize=12, fontweight='bold')
    ax.set_title(f'Top {top_n} Estados por Cantidad de Benefactores', 
                 fontsize=14, fontweight='bold', color=COLORES_TELETON['morado_profundo'])
    
    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)
    
    plt.tight_layout()
    return fig


def crear_grafico_antiguedad(df):
    orden = ['Nuevo (0-3 años)', 'Establecido (4-7 años)', 
             'Veterano (8-15 años)', 'Histórico (15+ años)']
    
    agg = df.groupby('segmento_antiguedad').agg({
        'satisfaccion_general': 'mean',
        'nps': 'mean',
        'indice_calidad_servicio': 'mean'
    }).reindex(orden)
    
    agg['indice_normalizado'] = agg['indice_calidad_servicio'] * 2
    
    fig, ax = plt.subplots(figsize=(12, 6))
    x = range(len(agg))
    
    ax.plot(x, agg['satisfaccion_general'], marker='o', linewidth=2.5,
            color=COLORES_TELETON['morado_profundo'], label='Satisfacción (0-10)')
    ax.plot(x, agg['indice_normalizado'], marker='s', linewidth=2.5,
            color=COLORES_TELETON['morado_medio'], label='Calidad (0-10 normalizado)')
    ax.plot(x, agg['nps'], marker='^', linewidth=2.5,
            color=COLORES_TELETON['amarillo'], label='NPS Promedio')
    
    ax.set_xticks(x)
    ax.set_xticklabels(agg.index, rotation=15, ha='right')
    ax.set_ylabel('Puntuación', fontsize=12, fontweight='bold')
    ax.set_title('Métricas por Antigüedad', 
                 fontsize=14, fontweight='bold', color=COLORES_TELETON['morado_profundo'])
    ax.set_ylim(0, 10)
    ax.legend(loc='best')
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)
    
    plt.tight_layout()
    return fig


def crear_heatmap_correlacion(df):
    vars_analisis = VARS_SATISFACCION + ['indice_calidad_servicio']
    corr = df[vars_analisis].corr()
    
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='YlGnBu',
                square=True, linewidths=1,
                cbar_kws={'label': 'Correlación'}, ax=ax)
    
    labels = ['Satisfacción', 'NPS', 'Calidad Percibida', 'Transparencia', 'Índice Calidad']
    ax.set_xticklabels(labels, rotation=45, ha='right')
    ax.set_yticklabels(labels, rotation=0)
    ax.set_title('Correlación: Satisfacción y Calidad', 
                 fontsize=14, fontweight='bold', color=COLORES_TELETON['morado_profundo'], pad=20)
    
    plt.tight_layout()
    return fig
'''

# Guardar módulo
with open('../streamlit/teleton_utils.py', 'w', encoding='utf-8') as f:
    f.write(codigo_modulo)

print("✅ Módulo teleton_utils.py creado en streamlit/")

## 10. Resumen del Notebook

In [ ]:
print("="*70)
print("RESUMEN: Preparación de Dashboard Streamlit")
print("="*70)
print("\n📦 FUNCIONES CREADAS:")
print("   ✅ 6 funciones para calcular KPIs")
print("   ✅ 1 función de segmentación NPS")
print("   ✅ 3 funciones de agregación por dimensión")
print("   ✅ 8 funciones de visualización con paleta Teletón")
print("\n📊 DATOS EXPORTADOS:")
print("   ✅ agregacion_giro.csv")
print("   ✅ agregacion_estado.csv")
print("   ✅ agregacion_antiguedad.csv")
print("   ✅ segmentos_nps.csv")
print("   ✅ kpis.csv")
print("\n🐍 MÓDULO PYTHON:")
print("   ✅ teleton_utils.py (listo para importar en Streamlit)")
print("\n🎯 PRÓXIMO PASO:")
print("   👉 Crear la aplicación Streamlit (app.py)")
print("   👉 Integrar todas las funciones del módulo")
print("   👉 Crear dashboard interactivo completo")
print("="*70)